In [ ]:
from queue import Queue
import random
from threading import Thread
import time


def producer(queue: Queue[int | None]):
    for i in range(10):
        time.sleep(random.uniform(0, 0.5))
        queue.put(i)
        print("生产者：", i)
    queue.put(None)


def consumer(queue: Queue[int | None]):
    while True:
        try:
            data = queue.get()
            if data is None:
                break
            time.sleep(random.uniform(0.5, 1.0))
            print(f"[consumer] 消费 {data}")
        finally:
            queue.task_done()


# 防止请求洪峰把内存打爆
queue = Queue(maxsize=3)
t1 = Thread(target=producer, args=(queue,))
t2 = Thread(target=consumer, args=(queue,))

t1.start()
t2.start()

t1.join()
t2.join()
print("所有任务完成")


生产者： 0
生产者： 1
生产者： 2
[consumer] 消费 0
生产者： 3
生产者： 4
[consumer] 消费 1
生产者： 5
[consumer] 消费 2
生产者： 6
[consumer] 消费 3
生产者： 7
[consumer] 消费 4
生产者： 8
[consumer] 消费 5
生产者： 9
[consumer] 消费 6
[consumer] 消费 7
[consumer] 消费 8
[consumer] 消费 9
所有任务完成


In [ ]:
from queue import Queue
import threading
import time
import random


def producer(q: Queue[str | None], data: list[str]):
    for d in data:
        q.put(d)
        print(f"【生产者】创建任务：{d}")
    q.put(None)
    print(f"【生产者】创建任务完成")


def consumer(q: Queue[str | None], results: list[str]):
    while True:
        try:
            task = q.get()
            if task is None:
                print("【消费者】消费任务完成")
                return results
            time.sleep(random.uniform(0.1, 0.5))
            results.append(f"done:{task}")
            print(f"【消费者】消费任务：{task}")
        finally:
            q.task_done()


def run_tasks(data: list[str], maxsize: int = 2):
    q: Queue[str] = Queue(maxsize)
    results: list[str] = []
    t1 = threading.Thread(target=producer, args=(q, data))
    t2 = threading.Thread(target=consumer, args=(q, results))

    t1.start()
    t2.start()

    t1.join()
    # ✅ 等待所有 put 的条目（包含 None）都被 task_done
    q.join()
    t2.join()

    assert results == ["done:A", "done:B", "done:C"], f"results：{results} != {data}"


data = ["A", "B", "C"]
QUEUE_MAXSIZE = 2
run_tasks(data, QUEUE_MAXSIZE)


【生产者】创建任务：A
【生产者】创建任务：B
【生产者】创建任务：C
【消费者】消费任务：A
【生产者】创建任务完成
【消费者】消费任务：B
【消费者】消费任务：C
【消费者】消费任务完成
